In [ ]:
import os
import pandas as pd
import logging
import re
from scripts.preprocesar_asistencias_fusagasuga import consolidar_archivos_fusagasuga_csv ,consolidar_asistencia_fusagasuga
from scripts.preprocesar_asistencias_fusagasuga import quitar_tildes,limpiar_espacios,convertir_a_formato_largo
from scripts.preprocesar_asistencias_fusagasuga import convertir_a_numero_mes_grado,agregar_bimestre_con_dias,mapear_estado_asistencia

1. Procesar cada hoja de asistencia por mes sede Fusagasuga

In [ ]:
# Archivo 1
consolidar_asistencia_fusagasuga("asistencia/1- Febrero.xlsx", "Febrero", 2024, "asistencia_consolidada_febrero.csv")
# Archivo 2
consolidar_asistencia_fusagasuga("asistencia/2 - Marzo.xlsx", "Marzo", 2024, "asistencia_consolidada_marzo.csv")
# Archivo 3
consolidar_asistencia_fusagasuga("asistencia/3 - Abril.xlsx", "Abril", 2024, "asistencia_consolidada_abril.csv")
# Archivo 4
consolidar_asistencia_fusagasuga("asistencia/4 - Mayo.xlsx", "Mayo", 2024, "asistencia_consolidada_mayo.csv")
# Archivo 5
consolidar_asistencia_fusagasuga("asistencia/5 - Junio.xlsx", "Junio", 2024, "asistencia_consolidada_junio.csv")
# Archivo 6
consolidar_asistencia_fusagasuga("asistencia/6 - Julio.xlsx", "Julio", 2024, "asistencia_consolidada_julio.csv")
# Archivo 7
consolidar_asistencia_fusagasuga("asistencia/7 - Agosto.xlsx", "Agosto", 2024, "asistencia_consolidada_agosto.csv")



2. Consolidar todos los meses procesados de Fusagasuga

In [9]:
df_consolidado = consolidar_archivos_fusagasuga_csv("./asistencia_csv")

Leyendo archivo: ./asistencia_csv\asistencia_consolidada_abril.csv
Leyendo archivo: ./asistencia_csv\asistencia_consolidada_agosto.csv
Leyendo archivo: ./asistencia_csv\asistencia_consolidada_febrero.csv
Leyendo archivo: ./asistencia_csv\asistencia_consolidada_julio.csv
Leyendo archivo: ./asistencia_csv\asistencia_consolidada_junio.csv
Leyendo archivo: ./asistencia_csv\asistencia_consolidada_marzo.csv
Leyendo archivo: ./asistencia_csv\asistencia_consolidada_mayo.csv
Consolidación completada. Archivo guardado como 'consolidado_asistencia_final_2024.csv'.


3. normalizar nombres estudiantes

In [10]:

df_consolidado["estudiante"]=df_consolidado["Nombre del estudiante"].apply(quitar_tildes)
df_consolidado["estudiante"]=df_consolidado["estudiante"].apply(limpiar_espacios)
df_consolidado.drop("Nombre del estudiante", axis=1, inplace=True)


df_consolidado.head()

,N°,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,Mes,Grado,Año,estudiante
0,1.0,A,NaN,NaN,A,NaN,NaN,NaN,A,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Abril,Quinto,2024,Avila Gonzalez Mariana
1,2.0,A,NaN,NaN,FJ,NaN,NaN,NaN,A,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Abril,Quinto,2024,Barrero Duarte Maria Paula
2,3.0,A,NaN,NaN,A,NaN,NaN,NaN,A,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Abril,Quinto,2024,Cardozo Parra Juan Angel
3,4.0,A,NaN,NaN,A,NaN,NaN,NaN,A,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Abril,Quinto,2024,Heung Garcia Maiya
4,5.0,FJ,NaN,NaN,FJ,NaN,NaN,NaN,A,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Abril,Quinto,2024,Leon Ceballos Martin


4. Reestructurar al formato largo
Para que cada fila represente un estudiante y un día, usa melt:

In [12]:
# Reestructurar al formato largo
id_vars = ["N°", "Mes", "Grado", "Año", "estudiante"]
df_consolidado_largo= convertir_a_formato_largo(df_consolidado,id_vars,1,31)


In [13]:
df_consolidado_largo.head(10)

,N°,Mes,Grado,Año,estudiante,Día,Estado
0,1.0,Abril,Quinto,2024,Avila Gonzalez Mariana,1,A
1,2.0,Abril,Quinto,2024,Barrero Duarte Maria Paula,1,A
2,3.0,Abril,Quinto,2024,Cardozo Parra Juan Angel,1,A
3,4.0,Abril,Quinto,2024,Heung Garcia Maiya,1,A
4,5.0,Abril,Quinto,2024,Leon Ceballos Martin,1,FJ
5,6.0,Abril,Quinto,2024,Lizarazo Mendoza Daniel,1,A
6,7.0,Abril,Quinto,2024,Lopez Ibanez Samuel,1,A
7,8.0,Abril,Quinto,2024,Luna Suarez Michelle Salome,1,A
8,9.0,Abril,Quinto,2024,Mosquera Bosistow Yamid Jeronimo,1,A
9,10.0,Abril,Quinto,2024,Novoa Ortiz Samuel Fernando,1,A


5. Mapear valores categóricos.
Convierte Mes y Dia a numero y agrega columna de bimestre segun calendario FICC 2024. 


In [14]:
df_consolidado_numerico=convertir_a_numero_mes_grado(df_consolidado_largo)
df_consolidado_numerico=agregar_bimestre_con_dias(df_consolidado_numerico)
df_consolidado_numerico["Estado"].fillna("A", inplace=True)
df_consolidado_numerico.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26195 entries, 0 to 26194
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   N°          26133 non-null  float64
 1   Mes         26195 non-null  object 
 2   Grado       26195 non-null  object 
 3   Año         26195 non-null  int64  
 4   estudiante  26195 non-null  object 
 5   Día         26195 non-null  int64  
 6   Estado      26195 non-null  object 
 7   Mes_Num     26195 non-null  int64  
 8   Grado_Num   26195 non-null  int64  
 9   Bimestre    26195 non-null  object 
dtypes: float64(1), int64(4), object(5)
memory usage: 2.0+ MB


C:\Users\Cathe\AppData\Local\Temp\ipykernel_23704\1112361608.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_consolidado_numerico["Estado"].fillna("A", inplace=True)


6. Convertir Estado en valor numerico. Convetir null a un valor en Estado. Eliminar columna N°

In [ ]:
df_consolidado_numerico=mapear_estado_asistencia(df_consolidado_numerico)
df_consolidado.drop("N°", axis=1, inplace=True)

df_consolidado_numerico["Sede"]="Fusagasugá"


,N°,Mes,Grado,Año,estudiante,Día,Estado,Mes_Num,Grado_Num,Bimestre,Estado_Num,Sede
0,1.0,Abril,Quinto,2024,Avila Gonzalez Mariana,1,A,4,5,I,1.0,Fusagasugá
1,2.0,Abril,Quinto,2024,Barrero Duarte Maria Paula,1,A,4,5,I,1.0,Fusagasugá
2,3.0,Abril,Quinto,2024,Cardozo Parra Juan Angel,1,A,4,5,I,1.0,Fusagasugá
3,4.0,Abril,Quinto,2024,Heung Garcia Maiya,1,A,4,5,I,1.0,Fusagasugá
4,5.0,Abril,Quinto,2024,Leon Ceballos Martin,1,FJ,4,5,I,0.5,Fusagasugá
5,6.0,Abril,Quinto,2024,Lizarazo Mendoza Daniel,1,A,4,5,I,1.0,Fusagasugá
6,7.0,Abril,Quinto,2024,Lopez Ibanez Samuel,1,A,4,5,I,1.0,Fusagasugá
7,8.0,Abril,Quinto,2024,Luna Suarez Michelle Salome,1,A,4,5,I,1.0,Fusagasugá
8,9.0,Abril,Quinto,2024,Mosquera Bosistow Yamid Jeronimo,1,A,4,5,I,1.0,Fusagasugá
9,10.0,Abril,Quinto,2024,Novoa Ortiz Samuel Fernando,1,A,4,5,I,1.0,Fusagasugá


In [26]:
df_consolidado_numerico['estudiante'] = df_consolidado_numerico['estudiante'].str.upper()
df_consolidado_numerico.head(10)




,N°,Mes,Grado,Año,estudiante,Día,Estado,Mes_Num,Grado_Num,Bimestre,Estado_Num,Sede
0,1.0,Abril,Quinto,2024,AVILA GONZALEZ MARIANA,1,A,4,5,I,1.0,Fusagasugá
1,2.0,Abril,Quinto,2024,BARRERO DUARTE MARIA PAULA,1,A,4,5,I,1.0,Fusagasugá
2,3.0,Abril,Quinto,2024,CARDOZO PARRA JUAN ANGEL,1,A,4,5,I,1.0,Fusagasugá
3,4.0,Abril,Quinto,2024,HEUNG GARCIA MAIYA,1,A,4,5,I,1.0,Fusagasugá
4,5.0,Abril,Quinto,2024,LEON CEBALLOS MARTIN,1,FJ,4,5,I,0.5,Fusagasugá
5,6.0,Abril,Quinto,2024,LIZARAZO MENDOZA DANIEL,1,A,4,5,I,1.0,Fusagasugá
6,7.0,Abril,Quinto,2024,LOPEZ IBANEZ SAMUEL,1,A,4,5,I,1.0,Fusagasugá
7,8.0,Abril,Quinto,2024,LUNA SUAREZ MICHELLE SALOME,1,A,4,5,I,1.0,Fusagasugá
8,9.0,Abril,Quinto,2024,MOSQUERA BOSISTOW YAMID JERONIMO,1,A,4,5,I,1.0,Fusagasugá
9,10.0,Abril,Quinto,2024,NOVOA ORTIZ SAMUEL FERNANDO,1,A,4,5,I,1.0,Fusagasugá


7. Hacer merge con numero de documento para facilitar cruzar la  inforacion con otras variables

In [29]:
# Leer el archivo CSV en un DataFrame.
df_estudiantes = pd.read_csv( "notas/Estudiantes.csv", delimiter=";")
df_estudiantes.head(-10)

,Id,Sede,Nivel,Orden Grado,Grado,Antigüedad,Género,Documento de Identificación,Primer Apellido,Segundo Apellido,Nombres,Fecha de nacimiento,estudiante
0,1,Girardot,Preescolar,-3,Prejardín,Nuevo,Femenino,1233700469,ACOSTA,SEGURA,MARGARET,11/4/2021,ACOSTA SEGURA MARGARET
1,2,Girardot,Preescolar,-3,Prejardín,Nuevo,Femenino,1015000870,SUAREZ,CASASBUENAS,ANA VERONCA,18/2/2021,SUAREZ CASASBUENAS ANA VERONCA
2,3,Girardot,Preescolar,-3,Prejardín,Antiguo,Masculino,1071063283,RODRIGUEZ,SALCEDO,JACOBO,6/5/2020,RODRIGUEZ SALCEDO JACOBO
3,4,Girardot,Preescolar,-3,Prejardín,Nuevo,Femenino,1012926444,CASTAÑEDA,SANTOS,MIA ISABEL,1/7/2020,CASTAÑEDA SANTOS MIA ISABEL
4,5,Girardot,Preescolar,-2,Jardín,Antiguo,Femenino,1070630315,LUNA,GARCIA,ANTONELLA,5/11/2019,LUNA GARCIA ANTONELLA
...,...,...,...,...,...,...,...,...,...,...,...,...,...
330,329,Fusagasugá,Secundaria,10,Décimo,Antiguo,Masculino,1171213074,VALIENTE,LOPEZ,LUIS ALEJANDRO,12/3/2009,VALIENTE LOPEZ LUIS ALEJANDRO
331,330,Fusagasugá,Secundaria,10,Décimo,Antiguo,Masculino,1069733865,VELASQUEZ,SIMBAQUEBA,JOHAN JHOSEPH,30/10/2008,VELASQUEZ SIMBAQUEBA JOHAN JHOSEPH
332,344,Fusagasugá,Secundaria,11,Undécimo,Antiguo,Masculino,1014990714,BERNAL,BENAVIDES,JHON ESTEBAN,17/8/2007,BERNAL BENAVIDES JHON ESTEBAN
333,333,Fusagasugá,Secundaria,11,Undécimo,Antiguo,Masculino,1070465326,DIAZ,FORERO,JULIAN FELIPE,23/6/2007,DIAZ FORERO JULIAN FELIPE


In [32]:

df_estudiantes["estudiante"]=df_estudiantes["estudiante"].apply(quitar_tildes)
df_estudiantes["estudiante"]=df_estudiantes["estudiante"].apply(limpiar_espacios)

In [33]:
# Realizar el merge
df_merge = pd.merge(df_consolidado_numerico, df_estudiantes, on="estudiante", how="left")

In [34]:
df_merge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26195 entries, 0 to 26194
Data columns (total 24 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   N°                           26133 non-null  float64
 1   Mes                          26195 non-null  object 
 2   Grado_x                      26195 non-null  object 
 3   Año                          26195 non-null  int64  
 4   estudiante                   26195 non-null  object 
 5   Día                          26195 non-null  int64  
 6   Estado                       26195 non-null  object 
 7   Mes_Num                      26195 non-null  int64  
 8   Grado_Num                    26195 non-null  int64  
 9   Bimestre                     26195 non-null  object 
 10  Estado_Num                   26122 non-null  float64
 11  Sede_x                       26195 non-null  object 
 12  Id                           25575 non-null  float64
 13  Sede_y          

In [35]:
df_merge.to_csv('asistencia_fusagasuga_2024.csv', index=False)

In [ ]:
#se ejecuta proceso manual de revision de faltantes comparando com archivo de calificaciones y luego se lee la version final
df_editado=pd.read_csv('asistencia_fusagasuga_2024-editado.csv',delimiter=";")
df_editado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26195 entries, 0 to 26194
Data columns (total 24 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   N°                           26133 non-null  float64
 1   Mes                          26195 non-null  object 
 2   Grado_x                      26195 non-null  object 
 3   Año                          26195 non-null  int64  
 4   estudiante                   26195 non-null  object 
 5   Día                          26195 non-null  int64  
 6   Estado                       26195 non-null  object 
 7   Mes_Num                      26195 non-null  int64  
 8   Grado_Num                    26195 non-null  int64  
 9   Bimestre                     26195 non-null  object 
 10  Estado_Num                   26195 non-null  float64
 11  Sede_x                       26195 non-null  object 
 12  Id                           25575 non-null  float64
 13  Sede_y          

In [ ]:

df_editado.drop(columns=['Id', 'Sede_y', 'Nivel',"Orden Grado","Grado_y"], inplace=True)
df_editado.rename(columns={'Sede_x': 'Sede'}, inplace=True)
df_editado.rename(columns={'Grado_x': 'Grado'}, inplace=True)

In [ ]:
df_editado.drop(columns=['N°', 'Primer Apellido', 'Segundo Apellido',"Nombres","Fecha de nacimiento"], inplace=True)
df_editado.to_csv('asistencia_fusagasuga_2024-imputado.csv', index=False)
df_editado.info()